<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Soloviev/Kuzmin" data-toc-modified-id="Soloviev/Kuzmin-1">Soloviev/Kuzmin</a></span><ul class="toc-item"><li><span><a href="#Symmetrical-Josephson-vortex-interferometer-as-an-advanced-ballistic-single-shot-detector---APL-2014" data-toc-modified-id="Symmetrical-Josephson-vortex-interferometer-as-an-advanced-ballistic-single-shot-detector---APL-2014-1.1">Symmetrical Josephson vortex interferometer as an advanced ballistic single-shot detector - APL 2014</a></span></li></ul></li></ul></div>

# Imports


In [2]:
import sys
sys.path.append('/Users/caleb/Development/HelperFunctions/')
from plottingFunctions import *

init_notebook_mode(connected=True)
from IPython.display import JSON, Image
from scipy.ndimage import gaussian_filter1d
from scipy.signal import find_peaks_cwt, argrelmin, butter, lfilter 
from scipy.constants import h, e, hbar
from scipy import optimize

from subprocess import call

from sympy import plot_implicit, symbols, Eq, solve, sin, nsolve

import json

# Lit Review

## Soloviev/Kuzmin 

### Symmetrical Josephson vortex interferometer as an advanced ballistic single-shot detector - APL 2014

It appears that the main drawbacks in its operation come from relativistic effects of the fluxon dynamics. In the experiment,the authors used single annular JTL coupled to the qubit instead of a couple, measuring deviation of the fluxon rotation frequency. The measurement results show that this deviation does not depend on the measured magnetic field orientation (the current dipole polarity). The qualitative explanation is that the relativistic fluxon characteristic size becomes much smaller than the dipole length due to Lorenz contraction. Therefore, the total contribution of the successive scatterings at the dipole poles to the frequency shift is independent on the order of the poles. Since the fluxon being inside the coupling loop induces circulating current affecting the qubit, the contraction additionally enhance the back-action. While slow fluxon is obviously preferred to gain the time response, the corresponding bias current, acting as the fluxon driving force, appears to be unreasonable for the experiment.

# JTL Class

Need to define a class to store/compute relevent properties of a JTL given design parameters. Also want to output a .cir file.

Input parameters: 

- $I_c$ - Critical Current ($\mu$A)
- $L$ - JTL Inductance (pH)
- $C$ - Junction capactiance (fF)
- $a$ - Cell Size ($\mu$m)
- $w$ - Inductor width ($\mu$m)
- $t_i$ - Gap between ground and inductor ($\mu$m)
- $d$ - Electrode thickness ($\mu$m)
- $n$ - Number of JTL cells

Computed parameters:

- $\lambda_J$ - $\sqrt{L_j/L}$ - Josephson length (cells)
- $\omega_p$ - $\sqrt{2eI_c/\hbar C}$ - $c/\lambda_J$ - Plasma frequency
- $\beta^2$ - $(4e^2/\hbar)\sqrt{\frac{L}{C}}$ - Wave resistance/impedance
- $c$ - $1/\sqrt{LC}$ - Swihart Velocity (speed of wave) (cells/s)
- $m$ - $(2\hbar/e)^{3/2}\sqrt{I_c L}C$ - Mass of soliton
- $\epsilon$ - $$ - Energy of soliton

In [3]:
phi0 = h/2/e

class UJTL:
    
    def __init__(self, params):
        self.params = params
        
        # Electrical Properties
        self.Ic = params['Ic']
        self.L = params['L']
        self.C = params['C']
        
        
        # Physical Properties
        self.a = params['a']
        self.w = params['w']
        self.ti = params['ti']
        self.d = params['d']
        self.n = params['n']
        
        
        
    # Dependent Properties
    
    @property
    def lambda_j(self):
        return np.sqrt(hbar/(2*e*self.Ic*self.L))
    
    @property
    def wp(self):
        return self.c/self.lambda_j
    
    @property    
    def beta2(self):
        return (4*e**2/hbar)*np.sqrt(self.L/self.C)
    
    @property    
    def c(self):
        return 1/np.sqrt(self.L*self.C)
    
    @property    
    def m(self):
        return (2*hbar/e)**(3/2)*np.sqrt(self.Ic*self.L)*self.C
    
    @property    
    def eps(self):
        return (e**2*self.C)(self.lambda_j/4)
    
    @property
    def u(self):
        return self.c(self.beta**2 * self.lambda_j/8)
    # Methods
    
    def pscan_units(self):
        
        pass

In [4]:
params = {
    'Ic': 7e-6,
    'L': 4e-12,
    'C': 4e-15,
    
    'a': 10e-6,
    'w': 2e-6,
    'ti': .1e-6,
    'd': .1e-6,
    'n': 20
}

jtl = UJTL(params) 

In [5]:
jtl.lambda_j

3.4283793324635514

In [6]:
jtl.beta2

0.030789640476528865

In [7]:
jtl.Ic

7e-06

In [8]:
(10*1e3)/(.01)**2 * (np.pi*0.15e-6**2)

7.068583470577034e-06

# JPM Class

Input parameters: 

- $I_c$ - Critical Current ($\mu$A)
- $L_g$ - Gradiometer Inductance
- $L_b$ - Flux bias Inductance
- $M_b$ - Flux bias mutual
- $L_{JTL}$ - JTL Inductance
- $M_{JTL}$ - JTL mutual
- $C$ - Shunt Capacitance (pF)
- $d$ - Dipole Size / Gradiometer size ($\mu$m)
- $\tan{\delta} = 1/Q$ - Loss Tangent

Computed parameters:

- $L$ - Total JPM Inductance (pH)
- $\beta_L$ 
- $f_{01}$ - Max JPM Frequency (GHz)
- $T_1$ - $1/(\delta \omega_{01})$ - Decay rate of JPM (ns)


In [64]:
class JPM:
    
    def __init__(self, params):
        
        self.__dict__.update(params)
        
        
    @property
    def L(self):
        return self.Lg
    
    @property
    def C(self):
        return self.Cs
    
    @property
    def R(self):
        return 1/(self.tand * self.w01 * self.Cs)
    
    @property
    def betaL(self):
        # L_rf / L_J
        return self.L * self.Ic * 2 * np.pi / phi0
    
    @property
    def w01(self):
        return np.sqrt((1 + self.betaL)/(self.L * self.C))
    
    @property
    def f01(self):
        return self.w01/2/np.pi
    
    def freqs(self, fluxes):
        return np.sqrt((1 + self.betaL*np.cos(fluxes))/(self.L * self.C)) / 2 / np.pi
       
    @property
    def T1(self):
        return 1/(self.tand * 2 * np.pi * self.f01)

In [109]:
testparams = {
    'Ic': 1e-6,
    'Lg': 1.5e-9,
    'Lb': 10e-12,
    'Mb': 17e-12,
    'Ljtl': 80e-12,
    'Mjtl': 500e-12,
    'Cs': 2.5e-12,
    'd': 35e-6,
    'tand': 3e-3,
    'C_load': [1e-15, 3e-15]
}

jpm = JPM(testparams)

In [110]:
jpm.f01 * 1e-9

6.1271152307812446

In [111]:
jpm.betaL

4.55780238174959

In [115]:
np.sqrt(4.4/(jpm.Cs*jpm.Lg) * (1+4.4))/2/np.pi / 1e9

12.668573514388394